# ASR Assignment 2022-23

This notebook has been provided as a template to get you started on the assignment.  Feel free to use it for your development, or do your development directly in Python.

You can find a full description of the assignment [here](http://www.inf.ed.ac.uk/teaching/courses/asr/2022-23/coursework.pdf).

You are provided with two Python modules `observation_model.py` and `wer.py`.  The first was described in [Lab 3](https://github.com/ZhaoZeyu1995/asr_labs/blob/master/asr_lab3_4.ipynb).  The second can be used to compute the number of substitution, deletion and insertion errors between ASR output and a reference text.

It can be used as follows:

```python
import wer

my_refence = 'A B C'
my_output = 'A C C D'

wer.compute_alignment_errors(my_reference, my_output)
```

This produces a tuple $(s,d,i)$ giving counts of substitution,
deletion and insertion errors respectively - in this example (1, 0, 1).  The function accepts either two strings, as in the example above, or two lists.  Matching is case sensitive.

## Template code

Assuming that you have already made a function to generate an WFST, `create_wfst()` and a decoder class, `MyViterbiDecoder`, you can perform recognition on all the audio files as follows:


In [ ]:
!pip install tqdm
!pip install matplotlib

In [ ]:
import glob
import os
import wer
import observation_model
import math
import openfst_python as fst
import time
import wave
from tqdm import tqdm

# probabilities here for easy edit
self_loop = 0.1
transition = 1 - self_loop # this is a choice
final = 1

In [ ]:
class MyViterbiDecoder:
    
    NLL_ZERO = 1e10  # define a constant representing -log(0).  This is really infinite, but approximate
                     # it here with a very large number
    
    def __init__(self, f, audio_file_name):
        """Set up the decoder class with an audio file and WFST f
        """
        self.om = observation_model.ObservationModel()
        self.f = f
        
        if audio_file_name:
            self.om.load_audio(audio_file_name)
        else:
            self.om.load_dummy_audio()
        
        self.initialise_decoding()

        
    def initialise_decoding(self):
        """set up the values for V_j(0) (as negative log-likelihoods)
        
        """
        
        self.V = []   # stores likelihood along best path reaching state j
        self.B = []   # stores identity of best previous state reaching state j
        self.W = []   # stores output labels sequence along arc reaching j - this removes need for 
                      # extra code to read the output sequence along the best path
        
        for t in range(self.om.observation_length()+1):
            self.V.append([self.NLL_ZERO]*self.f.num_states())
            self.B.append([-1]*self.f.num_states())
            self.W.append([[] for i in range(self.f.num_states())])  #  multiplying the empty list doesn't make multiple
        
        # The above code means that self.V[t][j] for t = 0, ... T gives the Viterbi cost
        # of state j, time t (in negative log-likelihood form)
        # Initialising the costs to NLL_ZERO effectively means zero probability    
        
        # give the WFST start state a probability of 1.0   (NLL = 0.0)
        self.V[0][self.f.start()] = 0.0
        
        # some WFSTs might have arcs with epsilon on the input (you might have already created 
        # examples of these in earlier labs) these correspond to non-emitting states, 
        # which means that we need to process them without stepping forward in time.  
        # Don't worry too much about this!  
        self.traverse_epsilon_arcs(0)        
        
    def traverse_epsilon_arcs(self, t):
        """Traverse arcs with <eps> on the input at time t
        
        These correspond to transitions that don't emit an observation
        
        We've implemented this function for you as it's slightly trickier than
        the normal case.  You might like to look at it to see what's going on, but
        don't worry if you can't fully follow it.
        
        """
        
        states_to_traverse = list(self.f.states()) # traverse all states
        while states_to_traverse:
            
            # Set i to the ID of the current state, the first 
            # item in the list (and remove it from the list)
            i = states_to_traverse.pop(0)   
        
            # don't bother traversing states which have zero probability
            if self.V[t][i] == self.NLL_ZERO:
                    continue
        
            for arc in self.f.arcs(i):
                
                if arc.ilabel == 0:     # if <eps> transition
                  
                    j = arc.nextstate   # ID of next state  
                
                    if self.V[t][j] > self.V[t][i] + float(arc.weight):
                        
                        # this means we've found a lower-cost path to
                        # state j at time t.  We might need to add it
                        # back to the processing queue.
                        self.V[t][j] = self.V[t][i] + float(arc.weight)
                        
                        # save backtrace information.  In the case of an epsilon transition, 
                        # we save the identity of the best state at t-1.  This means we may not
                        # be able to fully recover the best path, but to do otherwise would
                        # require a more complicated way of storing backtrace information
                        self.B[t][j] = self.B[t][i] 
                        
                        # and save the output labels encountered - this is a list, because
                        # there could be multiple output labels (in the case of <eps> arcs)
                        if arc.olabel != 0:
                            self.W[t][j] = self.W[t][i] + [arc.olabel]
                        else:
                            self.W[t][j] = self.W[t][i]
                        
                        if j not in states_to_traverse:
                            states_to_traverse.append(j)

    
    def forward_step(self, t, pruning_threshold):
        best_cost = self.NLL_ZERO # to be used in pruning
        nr_forward_comps = 0
        for i in self.f.states():
            
            if not self.V[t-1][i] == self.NLL_ZERO:   # no point in propagating states with zero probability
                
                for arc in self.f.arcs(i):
                    
                    if arc.ilabel != 0: # <eps> transitions don't emit an observation
                        nr_forward_comps += 1
                        j = arc.nextstate
                        
                        tp = float(arc.weight)  # transition prob
                        ep = -self.om.log_observation_probability(self.f.input_symbols().find(arc.ilabel), t)  # emission negative log prob
                        prob = tp + ep + self.V[t-1][i] # they're logs
                        
                        if pruning_threshold != 0:
                            if prob > best_cost - math.log(pruning_threshold): 
                                continue
                            
                        elif prob < self.V[t][j]:
                            self.V[t][j] = prob
                            self.B[t][j] = i
                            
                            best_cost = prob # best path to this specific state at time t
                            
                            # store the output labels encountered too
                            if arc.olabel !=0:
                                self.W[t][j] = [arc.olabel]
                            else:
                                self.W[t][j] = []
                        
        return nr_forward_comps
                            
    
    def finalise_decoding(self):
        """ this incorporates the probability of terminating at each state
        """
        
        for state in self.f.states():
            final_weight = float(self.f.final(state))
            if self.V[-1][state] != self.NLL_ZERO:
                if final_weight == math.inf:
                    self.V[-1][state] = self.NLL_ZERO  # effectively says that we can't end in this state
                else:
                    self.V[-1][state] += final_weight
                    
        # get a list of all states where there was a path ending with non-zero probability
        finished = [x for x in self.V[-1] if x < self.NLL_ZERO]
        if not finished:  # if empty
            print("No path got to the end of the observations.")
        
        
    def decode(self, pruning_threshold=0):
        start_time = time.time()
        
        self.initialise_decoding()
        t = 1
        tot_forwards = 0
        while t <= self.om.observation_length():
            n = self.forward_step(t, pruning_threshold)
            tot_forwards += n
            self.traverse_epsilon_arcs(t)
            t += 1
        self.finalise_decoding()
        end_time = time.time()
        return ((end_time - start_time), tot_forwards)
    
    def backtrace(self):
        start_time = time.time()
        best_final_state = self.V[-1].index(min(self.V[-1])) # argmin
        best_state_sequence = [best_final_state]
        best_out_sequence = []
        
        t = self.om.observation_length()   # ie T
        j = best_final_state
        
        while t >= 0:
            i = self.B[t][j]
            best_state_sequence.append(i)
            best_out_sequence = self.W[t][j] + best_out_sequence  # computer scientists might like
                                                                                # to make this more efficient!

            # continue the backtrace at state i, time t-1
            j = i  
            t-=1
            
        best_state_sequence.reverse()
        
        # convert the best output sequence from FST integer labels into strings
        best_out_sequence = ' '.join([ self.f.output_symbols().find(label) for label in best_out_sequence])
        end_time = time.time()
        
        return (best_state_sequence, best_out_sequence, (end_time - start_time))

In [ ]:
def parse_lexicon(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            lex[line[0]] = line[1:]  # first field the word, the rest is the phones
    return lex

In [ ]:
def parse_lexicon_improved(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    count = 0
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            
            if line[0] not in lex:
                lex[line[0]] = line[1:]  # first field the word, the rest is the phones
                count = 0
            else:
                count += 1
                lex[line[0]+str(count)] = line[1:]
    return lex

In [ ]:
def generate_symbol_tables(lexicon, n=3):
    '''
    Return word, phone and state symbol tables based on the supplied lexicon
        
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
        n (int): number of states for each phone HMM
        
    Returns:
        word_table (fst.SymbolTable): table of words
        phone_table (fst.SymbolTable): table of phones
        state_table (fst.SymbolTable): table of HMM phone-state IDs
    '''
    
    state_table = fst.SymbolTable()
    phone_table = fst.SymbolTable()
    word_table = fst.SymbolTable()
    
    # add empty <eps> symbol to all tables
    state_table.add_symbol('<eps>')
    phone_table.add_symbol('<eps>')
    word_table.add_symbol('<eps>')
    
    for word, phones  in lexicon.items():
        
        word_table.add_symbol(word)
        
        for p in phones: # for each phone
            
            phone_table.add_symbol(p)
            for i in range(1,n+1): # for each state 1 to n
                state_table.add_symbol('{}_{}'.format(p, i))
        
    # add silence symbols to state table as well, for task 2
    for i in range(6):
        state_table.add_symbol("sil_" + str(i))
              
    return word_table, phone_table, state_table

lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)
print(lex)

In [ ]:
def generate_phone_wfst(f, start_state, phone, n, word, is_final_phone):
    """
    Generate a WFST representating an n-state left-to-right phone HMM
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assmed to exist already
        phone (str): the phone label 
        n (int): number of states for each phone HMM
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    
    for i in range(1, n+1):
        
        in_label = state_table.find('{}_{}'.format(phone, i))
        
        sl_weight = fst.Weight('log', -math.log(self_loop))  # weight for self-loop
        # self-loop back to current state
        f.add_arc(current_state, fst.Arc(in_label, 0, sl_weight, current_state))
        
        # transition to next state
        out_label = 0
        
        if is_final_phone:
            if i == n:
                out_label = word_table.find(word)
            
        next_state = f.add_state()
        next_weight = fst.Weight('log', -math.log(transition)) # weight to next state
        f.add_arc(current_state, fst.Arc(in_label, out_label, next_weight, next_state))    
       
        current_state = next_state
        
    return current_state

In [ ]:
def generate_silence_wfst(f, start_state):
    """
    Generate a WFST representing the 'silence state' from the coursework description, 
    essentially copying what was described there
    """
    current_state = start_state
    
    sl_weight = fst.Weight('log', -math.log(self_loop))
    next_weight = fst.Weight('log', -math.log(transition)) # weight to next state
    
    # state 1
    in_label = state_table.find("sil_1")
    f.add_arc(current_state, fst.Arc(in_label, 0, sl_weight, current_state))
    
    state_2 = f.add_state()
    f.add_arc(current_state, fst.Arc(in_label, 0, next_weight, state_2))
    
    # state 2,3,4
    state_3 = f.add_state()
    state_4 = f.add_state()
    
    ergodic_states = [state_2, state_3, state_4]
    
    i = 2
    for state in ergodic_states:
        current_state = state
        
        # self loop arc
        in_label = state_table.find("sil_" + str(i))
        f.add_arc(current_state, fst.Arc(in_label, 0, sl_weight, current_state))
        
        remaining_states = [s for s in ergodic_states if s != state]
        
        for next_state in remaining_states:
            f.add_arc(current_state, fst.Arc(in_label, 0, next_weight, next_state))
        
        i += 1
    
    # state 5
    current_state = state_4
    
    in_label = state_table.find("sil_5")
    
    state_5 = f.add_state()
    f.add_arc(current_state, fst.Arc(in_label, 0, next_weight, state_5))
    
    f.add_arc(state_5, fst.Arc(in_label, 0, sl_weight, state_5))
    
    f.set_final(state_5, fst.Weight("log",-math.log(final)))
    
    return state_5
    
f = fst.Fst('log')
start = f.add_state()
f.set_start(start)

last_state = generate_silence_wfst(f, start)

f.set_input_symbols(state_table)
f.set_output_symbols(word_table)

from subprocess import check_call
from IPython.display import Image

f.draw('tmp.dot', portrait=True)
check_call(['dot','-Tpng','-Gdpi=200','tmp.dot','-o','tmp.png'])
Image(filename='tmp.png')

In [ ]:
def generate_word_wfst(f, start_state, word):
    """ Generate a WFST for any word in the lexicon, composed of 3-state phone WFSTs.
        This will currently output phone labels.  
        Exercise: could you modify this function and the one above to output a single word label instead?
    
    Args:
        word (str): the word to generate
        
    Returns:
        the constructed WFST
    
    """
    current_state = start_state
    
    base_word = ""
    for char in word:
        if not char.isnumeric():
            base_word += char
            
#     print((word, base_word))
    
    # iterate over all the phones in the word
    for i in range(len(lex[word])):
        phone = lex[word][i]
          
        current_state = generate_phone_wfst(f, current_state, phone, 3, base_word, i==len(lex[word])-1)
    
    return current_state

In [ ]:
# unigram
total_word_occurrences = 0
for t_file in glob.glob('/group/teaching/asr/labs/recordings/*.txt'):
    transcription = open(t_file, 'r').readline().strip().split()
    total_word_occurrences += len(transcription)
    

word_probs = {}
for word in lex.keys():
    this_word_occs = 0
    
    for t_file in glob.glob('/group/teaching/asr/labs/recordings/*.txt'):
        transcription = open(t_file, 'r').readline().strip().split()
        this_word_occs += transcription.count(word)

    word_probs[word] = this_word_occs / total_word_occurrences

print(word_probs)

In [ ]:
def generate_word_branch_wfst(word_list):
    f = fst.Fst("log")
    start_state = f.add_state()
    f.set_start(start_state)
    
    word_prob = 1/len(word_list) #standard
    
    
    for word in word_list:
        
#         word_prob = word_probs[word] #unigram
        
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, fst.Weight("log",-math.log(word_prob)), current_state))

        current_state = generate_word_wfst(f, current_state, word)
        f.set_final(current_state, fst.Weight("log",-math.log(final)))
    
        f.add_arc(current_state, fst.Arc(0, 0, fst.Weight("log",-math.log(1)), start_state))
    
    return f

In [ ]:
def generate_word_branch_wfst_silence(word_list):
    f = fst.Fst("log")
    start_state = f.add_state()
    f.set_start(start_state)
    n = len(word_list) + 1
    
    for word in word_list:
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, fst.Weight("log",-math.log(1/n)), current_state))

        current_state = generate_word_wfst(f, current_state, word)
        f.set_final(current_state, fst.Weight("log",-math.log(final)))
    
        f.add_arc(current_state, fst.Arc(0, 0, fst.Weight("log",-math.log(final)), start_state))
    
    silence_start = f.add_state()
    f.add_arc(start_state, fst.Arc(0, 0, fst.Weight("log",-math.log(1/n)), silence_start))
    
    silence_end = generate_silence_wfst(f, silence_start)
    f.add_arc(silence_end, fst.Arc(0, 0, fst.Weight("log",-math.log(1)), start_state))
    return f

In [ ]:
def bigram_prob(before: str, after: str):
    bigram_probs = {}
    
    for t_file in glob.glob('/group/teaching/asr/labs/recordings/*.txt'):
        transcription = ['<s>'] + open(t_file, 'r').readline().strip().split() + ['</s>']
        
        for i in range(len(transcription)-1):
            word = transcription[i]
            next_word = transcription[i+1]
            
            if word != before:
                continue
            
            if next_word in bigram_probs:
                bigram_probs[next_word] += 1
            else:
                bigram_probs[next_word] = 1
    
    print(bigram_probs)
    if after not in bigram_probs:
        return 0
    return (bigram_probs[after] / sum(bigram_probs.values())) 

bigram_prob("peck", "of")

In [ ]:
def generate_word_branch_wfst_bigram(word_list):
    f = fst.Fst("log")
    start_state = f.add_state()
    f.set_start(start_state)
    
    end_state = f.add_state()
    f.set_final(end_state, fst.Weight("log",-math.log(final)))
    
    word_start_end = []
    
    for word in word_list:
        word_start = f.add_state()
        if bigram_prob('<s>', word) != 0:
            f.add_arc(start_state, fst.Arc(0, 0, fst.Weight("log",-math.log(bigram_prob('<s>', word))), word_start))
        
        word_end = generate_word_wfst(f, word_start, word)
        
        if bigram_prob(word, '</s>') != 0:
            f.add_arc(word_end, fst.Arc(0, 0, fst.Weight("log",-math.log(bigram_prob(word, '</s>'))), end_state))
        
        word_start_end.append((word, word_start, word_end))
    
    for (word, word_start, word_end) in word_start_end:
        for (word2, word2_start, word2_end) in word_start_end:
            
            if bigram_prob(word, word2) != 0:
                f.add_arc(word_end, fst.Arc(0, 0, fst.Weight("log",-math.log(bigram_prob(word, word2))), word2_start))
    
    return f

In [ ]:
def create_wfst():
#     f = generate_word_branch_wfst([k for k in lex.keys()])
    f = generate_word_branch_wfst_bigram([k for k in lex.keys()])
#     f = generate_word_branch_wfst_bigram([k for k in lex.keys()][:3])

#     f = fst.determinize(f)
    
#     f = generate_word_branch_wfst_silence([k for k in lex.keys()])
    f.set_input_symbols(state_table)
    f.set_output_symbols(word_table)
    
    return f

from subprocess import check_call
from IPython.display import Image
f = create_wfst()
f.draw('tmp.dot', portrait=True)
check_call(['dot','-Tpng','-Gdpi=200','tmp.dot','-o','tmp.png'])
Image(filename='tmp.png')

In [ ]:
def nr_states_arcs(f):
    nr_states = 0
    nr_arcs = 0
    
    for state in f.states():
        nr_states += 1
        for arc in f.arcs(state):
            nr_arcs += 1

    return nr_states,nr_arcs

In [ ]:
def read_transcription(wav_file):
    """
    Get the transcription corresponding to wav_file.
    """
    
    transcription_file = os.path.splitext(wav_file)[0] + '.txt'
    
    with open(transcription_file, 'r') as f:
        transcription = f.readline().strip()
    
    return transcription


f = create_wfst()

substitutions = 0
deletions = 0
insertions = 0
tot_word_count = 0
tot_decode_time = 0
tot_backtrace_time = 0
tot_forwards = 0
tot_frames = 0
tot_actual_phones = 0

nr_states, nr_arcs = nr_states_arcs(f)
print(nr_states, nr_arcs)

for wav_file in tqdm(glob.glob('/group/teaching/asr/labs/recordings/*.wav')):    # replace path if using your own
                                                                           # audio files
    
    decoder = MyViterbiDecoder(f, wav_file)
    
#     audio_file = wave.open(wav_file, 'r')
#     tot_frames += audio_file.getnframes()
#     print(audio_file.getnframes())
    
    
    (decode_time, nr_forwards) = decoder.decode()
    (state_path, words, backtrace_time) = decoder.backtrace()  # you'll need to modify the backtrace() from Lab 4
                                               # to return the words along the best path

    transcription = read_transcription(wav_file)
    
#     for word in transcription.split(' '):
#         phones = lex[word]
# #         print(phones)
#         tot_actual_phones += 3 * len(phones)
    
    error_counts = wer.compute_alignment_errors(transcription, words)
    word_count = len(transcription.split())
    
    substitutions += error_counts[0]
    deletions += error_counts[1]
    insertions += error_counts[2]
    tot_word_count += word_count
    tot_decode_time += decode_time
    tot_backtrace_time += backtrace_time
    tot_forwards += nr_forwards
    
print("Total word count: " + str(tot_word_count))
final_wer = ((substitutions + deletions + insertions) / tot_word_count) * 100
print("S = " + str(substitutions))
print("D = " + str(deletions))
print("I = " + str(insertions))

print("WER = " + str(final_wer))
print("Total decode and backtrace time " + str((tot_decode_time, tot_backtrace_time)))
print("No. of forwards " + str(tot_forwards))

print("Total number of frames: " + str(tot_frames))
print("Total number of phones in transcription " + str(tot_actual_phones))

In [ ]:
len(glob.glob('/group/teaching/asr/labs/recordings/*.wav'))

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# labels = (0.1, 0.2, 0.5, 0.8, 0.99)
# sdi_counts = {"S": np.array([605, 618, 607, 583, 601]), "D": np.array([19, 36, 56, 96, 203]), 
#               "I": np.array([2885, 1852, 1169, 921, 635])}
# width = 0.075

# fig, ax = plt.subplots(figsize=(10, 5))

# bottom = np.zeros(len(labels))  # Initialize the bottom of each bar stack

# for category, count in sdi_counts.items():
#     ax.bar(labels, count, width, label=category, bottom=bottom)
#     bottom += count

# ax.set_title("SDI counts with changing self-loop probability")
# ax.legend(loc="upper right")
# ax.set_xticks(labels)
# ax.set_xlabel("Self-loop probability")

# plt.show()